# Lab 3: Camera calibration using a planar pattern


## 1. Introduction

In this assignment we are going to find the camera calibration parameters using a planar pattern.  The method consists of taking several images of a known planar pattern in different positions.  By detecting the pattern in the images, we will be able to recover the camera internal parameters as well as its pose relative to the pattern.  This will enable us to draw the cameras in 3D and to project 3D objects onto the images as in Augmented Reality applications.

### 1.1 The pattern
The planar pattern can be any flat image, usually a chessboard.  For simplicity, instead of the chessboard, we are going to use a flat screen to display a texture-rich image.  The ORB/SIFT/SURF descriptor will be used to compute correspondences between the picture of the screen with the picture stored in the computer, as it has been done in previous assignments for computing homographies.

### 1.2 Taking the images
You will use the provided images in a first stage.  Later on, you are asked to take pictures of a flat pattern with any other camera you want to calibrate.

### 1.3 Goals and Requirements

The goal of the current lab assignment is to learn the following concepts:

- How to compute the internal camera parameters from a set of plane homographies.  We will use the Image of the Absolute Conic for this. 

- How to compute the camera position with respect to the calibration target.

- How to add simple virtual 3D objects on the images of the calibration target.


We will assume that we already know the following concepts:

- What is a homography, how to match ORB/SIFT/SURF keypoints between images and how to robustly compute a homography from point correspondences.

- What is the camera projection matrix and what are the internal and external camera parameters and their matrix form $P = K(R|t)$.

- You have an idea of what is the Image of the Absolute Conic and you know that its matrix is simply $w = (K K ^T)^{-1} = K^{-T} K^{-1}$.

In [ ]:
import numpy as np
import cv2
import random
import math
import sys
import matplotlib
from matplotlib import pyplot as plt
from operator import itemgetter
from utils import Ransac_DLT_homography, plot_camera, plot_image_origin
import plotly.graph_objects as go

## 2. Calibration with a planar pattern

### 2.1 Load the images and compute homographies

As in the previous session, we are going to compute homographies between images using ORB/SIFT/SURF matching and RANSAC. 

In [ ]:
template = cv2.imread('Data/template.jpg',cv2.IMREAD_GRAYSCALE)
images = []
N = 3 # 5
for i in range(1,N+1):
    m = cv2.imread("Data/calib{0}.jpg".format(i),cv2.IMREAD_GRAYSCALE)
    images.append(m)
    
# Initiate SIFT detector
sift = cv2.SIFT_create(4000)

# find the keypoints and descriptors
kpt, dest = sift.detectAndCompute(template,None)
kpi, desi = [], []
for m in images:
    kp, des = sift.detectAndCompute(m,None)
    kpi.append(kp)
    desi.append(des)


# Keypoint matching and homography estimation
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
H = []
for i in range(N):
    matches = bf.match(dest,desi[i])
    
    # Fit homography and remove outliers
    points1 = []
    points2 = []
    for m in matches:
        points1.append([kpt[m.queryIdx].pt[0], kpt[m.queryIdx].pt[1], 1])
        points2.append([kpi[i][m.trainIdx].pt[0], kpi[i][m.trainIdx].pt[1], 1])
        
    points1 = np.asarray(points1)
    points1 = points1.T
    points2 = np.asarray(points2)
    points2 = points2.T
    
    Hi, indices_inlier_matches = Ransac_DLT_homography(points1, points2, 3, 1000)
    H.append(Hi)
    
    # Show inlier matches
    inlier_matches = itemgetter(*indices_inlier_matches)(matches)
    img_12 = cv2.drawMatches(template,kpt,images[i],kpi[i],inlier_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.imshow(img_12)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()

### 2.2 Compute the Image of the Absolute Conic

Given the homographies we are now going to compute the Image of the Absolute conic (IAC).  Read carefully the theory in the provided file 'calibration.pdf'.

**Q1.** Complete the code below to compute the IAC from the homographies.

**Q2.** How many images are required to compute the IAC? What would you do if you had fewer images?

In [ ]:
# Compute the Image of the Absolute Conic
A = np.zeros((2*N, 6))

for i in range(N):
    h = H[i]
    A[2*i-1, :] = np.array([h[0,0]*h[0,1], h[0,0]*h[1,1] + h[1,0]*h[0,1], h[0,0]*h[2,1] + h[2,0]*h[0,1], h[1,0]*h[1,1], h[1,0]*h[2,1] + h[2,0]*h[1,1], h[2,0]*h[2,1] ])
    A[2*i, :] = np.array([h[0,0]**2, 2*h[0,0]*h[1,0], 2*h[0,0]*h[2,0], h[1,0]**2, 2*h[1,0]*h[2,0], h[2,0]**2])
    A[2*i, :] -= np.array([h[0,1]**2, 2*h[0,1]*h[1,1] , 2*h[0,1]*h[2,1], h[1,1]**2, 2*h[1,1]*h[2,1], h[2,1]**2])

# complete ... 
w = # ...

### 2.3 Compute the camera calibration matrix from the IAC

The IAC relates to the camera calibration matrix, $K$ as $\omega = K^{-T} K^{-1}$. Knowing $\omega$ we can get $K$ using the Cholesky factorization.

**Q3.** Write the code to compute the camera calibration from the IAC.

**Q4.** What result do you get? Can you interpret the values of $K$ in geometric terms? Which ones are related to the image size? How are they related? Do they make sense in your results?

**Q5.** Compute the field of view angle of the camera.

In [ ]:
# complete ... 
K = # ...

### 2.4 Compute camera position and orientation

Given the computed calibration $K$ and the homografies $H$, we can compute the 3D position and orientation of the camera with respect to the planar pattern.

**Q6.** Complete the code to compute the rotation and translation of each camera.

**Q7.** In which units are expressed the translation vector `ti`?

In [ ]:
# Compute camera position and orientation
import math
R = []
t = []
P = []

for i in range(N):
    # compute r1, r2, and t{i}
    h = H[i]
    r1 = # complete ...
    r2 = # complete ...
    ti = # complete ...
    
    # Solve the scale ambiguity by forcing r1 and r2 to be unit vectors.
    s = math.sqrt(np.linalg.norm(r1) * np.linalg.norm(r2)) * np.sign(ti[2])
    r1 = r1 / s
    r2 = r2 / s
    ti = ti / s
    t.append(ti)
    Ri = np.array([r1, r2, np.cross(r1,r2)])
    Ri = Ri.T
    
    # Ensure R is a rotation matrix
    #[U S V] = svd(R{i});
    U, d, Vt = np.linalg.svd(Ri)
    Ri = U @ np.identity(3) @ Vt
    R.append(Ri)
   
    # Pi = K * [Ri ti]
    A = np.zeros((3,4))
    A[:3,:3] = Ri
    A[:,3] = ti
    Pi = K @ A
    P.append(Pi)

The following code allows to visualize in 3D the different cameras corresponding to the different images and the frame (rectangular boundaries) of the planar pattern. The cameras are illustrated with a pyramid that encodes the estimated position (apex) and orientation (base).

In [ ]:
import plotly.graph_objects as go    

ny, nx = images[0].shape

fig = go.Figure()
for i in range(N):
    plot_camera(P[i], nx, ny, fig, "camera{0}".format(i))

plot_image_origin(nx/2, ny/2, fig, "image")
fig.show()

**Q8.** Complete the code below to plot a moving planar target in front of a static camera.

**Q9.** When taking the images, we did move the camera and the planar pattern was static. The last plot shows a moving pattern in front of a static camera.  What is the relation between the two situations?

In [ ]:
def plot_image_Rt(R,t,w, h, fig, legend):
    
    p1 = # to complete ...
    p2 = # to complete ...
    p3 = # to complete ...
    p4 = # to complete ...
    
    x = np.array([p1[0], p2[0], p3[0], p4[0], p1[0]])
    y = np.array([p1[1], p2[1], p3[1], p4[1], p1[1]])
    z = np.array([p1[2], p2[2], p3[2], p4[2], p1[2]])

    fig.add_trace(go.Scatter3d(x=x, y=z, z=-y, mode='lines',name=legend))
    
    return


fig = go.Figure()
A = np.zeros((3,4))
A[0,0]=A[1,1]=A[2,2]=1

plot_camera(K@A, nx, ny, fig, "camera")
for i in range(N):
    plot_image_Rt(R[i], t[i], nx/2, ny/2, fig, "image{0}".format(i))

fig.show()

### 2.5 Augmented reality

We now have the projection matrices, $P$, of each image with respect to the planar target's reference frame. We can use these projection matrices for projecting 3D points onto the images. Given a 3D point, we can project it using the projection matrix and plot the projection over the image.

**Q10.** Complete the code below to plot a cube in front of the calibration pattern on each image. The result should look as if a cube was attached to the pattern as the pattern moves from image to image.

In [ ]:
Th, Tw = template.shape
cube_corners = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 1, 1], [0, 1, 1], [0, 0, 1], [0, 0, 0], [1, 0, 0], [1, 0, 1], [0, 0, 1], [0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 1, 1], [0, 1, 1], [0, 1, 0]])

import numpy.matlib
offset = np.array([Tw/2, Th/2, -Tw/8])
M = cube_corners.shape[0]
X = (cube_corners - 0.5) * Tw/ 4 + np.matlib.repmat(offset, M, 1)

X = X.T
ones = np.ones(M)
Xh = np.stack((X[0,:], X[1,:], X[2,:], ones), axis=0)

line_color = (0, 255, 0)

for i in range(N):
    
    xp = # complete ...
    
    image = cv2.imread("Data/calib{0}.jpg".format(i+1),cv2.IMREAD_COLOR) 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_gray = image
    image_gray[:, :, 0] = gray[:, :]
    image_gray[:, :, 1] = image_gray[:, :, 0]
    image_gray[:, :, 2] = image_gray[:, :, 0]
    for j in range(M-1):
        image = cv2.line(image_gray, (xp[0,j],xp[1,j]), (xp[0,j+1],xp[1,j+1]), line_color, 4) 
    plt.imshow(image)
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()


**Q11.** Repeat all the process using your own images (you may use from 3 to 5 different views of a planar pattern). Comment the results and try to find an explanation in case it doesn't work.

### 2.6 Reducing the number of images

In case we have some knowledge about the camera parameters like the principal point, the aspect ratio or zero skew factor the number of images needed for calibration can be reduced.

**Q12.** Add a linear constraint to enforce skew 0.  See Zhang's paper (zhang98.pdf) [1]. What is the minimum number of views you need to calibrate the camera in this case?

**Q13.** Can you calibrate from a single image? How? Adapt your code to be able to calibrate from just a single image. Compare the results with the ones obtained when using more images. 

**Optional:** Using the last result you can create a video where the projected virtual cube automatically adapts to the changing point of view when you move the camera.

[1] Zhengyou Zhang. A flexible new technique for camera calibration, IEEE Transactions on pattern analysis and machine intelligence, 22(11), 1330-1334, 2000.